# 06 - Calculate crop fraction in point buffers

#### Description

The goal of this notebook is to calculate fraction, area and number of areas that exist in each point buffer.

It implements the following steps:

- determine the area and fraction each crop represents in the occurrence buffer
- determine and add to the occurrence table information about crops and pests that birds use for feeding

#### Inputs

- filtered set of eBird occurrences produced by step 03: `eBird_sel.csv`
- crop cover data at buffer, produced by step 04: `eppo_crop_pest_data.csv'`
- table with data of which pests of which crops are eaten by birds: `bird_prey_crop.csv`
   

#### Outputs

- table with the fraction of the crop in the buffer area

## 1. Read eBird data (occurrences)

This dataset contains:
- eBird data
- stats on crop cover at buffer of occurrence point

In [1]:
# import modules
import pandas as pd
import numpy as np
from pandas import DataFrame
import geopandas as gpd
from pyproj import Proj, CRS,transform
import matplotlib.pyplot as plt

In [2]:
# load eBird selected occurrence data
path = '../process_data/'
occ_file = path + 'eBird_sel.csv'

bird_data_name = pd.read_csv(occ_file, low_memory=False)


# ensure that dates are of type datetime
bird_data_name['OBSERVATION DATE'] = pd.to_datetime(bird_data_name['OBSERVATION DATE'])


In [3]:
bird_data_name.head()

,Unnamed: 0.1,Unnamed: 0,GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER_x,CATEGORY,TAXON CONCEPT ID_x,COMMON NAME,SCIENTIFIC NAME_x,SUBSPECIES COMMON NAME,...,Annual_crops,Permanent_crops,Proportion_invertebrates_diet,_merge,NativeToValCal,LOCALITY_y,LOCALITY ID_y,geometry_y,stats_cover,area_buff
0,0,0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,2022-06-02 18:22:45.475308,26451,species,avibase-603194D3,Bewick's Wren,Thryomanes bewickii,NaN,...,1.0,1.0,1.0,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
1,1,1926,URN:CornellLabOfOrnithology:EBIRD:OBS1446126014,2022-06-02 18:22:45.475308,11494,species,avibase-20C2214E,American Kestrel,Falco sparverius,NaN,...,0.0,1.0,0.3,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
2,2,2608,URN:CornellLabOfOrnithology:EBIRD:OBS1446126023,2022-06-02 18:22:45.475308,20505,species,avibase-151C2B3F,California Scrub-Jay,Aphelocoma californica,NaN,...,0.0,1.0,0.4,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
3,3,9726,URN:CornellLabOfOrnithology:EBIRD:OBS1446126018,2022-06-02 18:22:45.475308,11371,species,avibase-75600969,Northern Flicker,Colaptes auratus,NaN,...,1.0,1.0,1.0,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
4,4,10252,URN:CornellLabOfOrnithology:EBIRD:OBS1446126013,2022-06-02 18:22:45.475308,20762,species,avibase-9E9F2476,American Crow,Corvus brachyrhynchos,NaN,...,0.0,1.0,0.4,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3


In [4]:
bird_data_name['OBSERVATION DATE'].dt.month.max()

6

In [5]:
bird_data_name.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'GLOBAL UNIQUE IDENTIFIER',
       'LAST EDITED DATE', 'TAXONOMIC ORDER_x', 'CATEGORY',
       'TAXON CONCEPT ID_x', 'COMMON NAME', 'SCIENTIFIC NAME_x',
       'SUBSPECIES COMMON NAME', 'SUBSPECIES SCIENTIFIC NAME', 'EXOTIC CODE',
       'OBSERVATION COUNT', 'BREEDING CODE', 'BREEDING CATEGORY',
       'BEHAVIOR CODE', 'AGE/SEX', 'COUNTRY', 'COUNTRY CODE', 'STATE',
       'STATE CODE', 'COUNTY', 'COUNTY CODE', 'LOCALITY_x', 'LOCALITY ID_x',
       'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE',
       'TIME OBSERVATIONS STARTED', 'OBSERVER ID', 'SAMPLING EVENT IDENTIFIER',
       'PROTOCOL TYPE', 'PROTOCOL CODE', 'DURATION MINUTES',
       'EFFORT DISTANCE KM', 'EFFORT AREA HA', 'NUMBER OBSERVERS',
       'ALL SPECIES REPORTED', 'GROUP IDENTIFIER', 'APPROVED', 'REVIEWED',
       'REASON', 'geometry_x', 'TAXON CONCEPT ID_y', 'SCIENTIFIC NAME_y',
       'TAXONOMIC ORDER_y', 'ORDER1', 'Species2', 'Family2', 'Order2',
       'Avibase.ID2',

In [6]:
bird_data_name.head()

,Unnamed: 0.1,Unnamed: 0,GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER_x,CATEGORY,TAXON CONCEPT ID_x,COMMON NAME,SCIENTIFIC NAME_x,SUBSPECIES COMMON NAME,...,Annual_crops,Permanent_crops,Proportion_invertebrates_diet,_merge,NativeToValCal,LOCALITY_y,LOCALITY ID_y,geometry_y,stats_cover,area_buff
0,0,0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,2022-06-02 18:22:45.475308,26451,species,avibase-603194D3,Bewick's Wren,Thryomanes bewickii,NaN,...,1.0,1.0,1.0,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
1,1,1926,URN:CornellLabOfOrnithology:EBIRD:OBS1446126014,2022-06-02 18:22:45.475308,11494,species,avibase-20C2214E,American Kestrel,Falco sparverius,NaN,...,0.0,1.0,0.3,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
2,2,2608,URN:CornellLabOfOrnithology:EBIRD:OBS1446126023,2022-06-02 18:22:45.475308,20505,species,avibase-151C2B3F,California Scrub-Jay,Aphelocoma californica,NaN,...,0.0,1.0,0.4,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
3,3,9726,URN:CornellLabOfOrnithology:EBIRD:OBS1446126018,2022-06-02 18:22:45.475308,11371,species,avibase-75600969,Northern Flicker,Colaptes auratus,NaN,...,1.0,1.0,1.0,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3
4,4,10252,URN:CornellLabOfOrnithology:EBIRD:OBS1446126013,2022-06-02 18:22:45.475308,20762,species,avibase-9E9F2476,American Crow,Corvus brachyrhynchos,NaN,...,0.0,1.0,0.4,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3


## 2. Calculate area and cover percentage per crop

This section is to determine the fraction that each crop area represents in the buffer around each occurrence point. This is necessary to weight the total consumption of pest by birds in a specific crop to fraction that this crop represents.

The columns added to the eBird data table, for each occurrence point are:
- total: total area
- count_area: count of all different land uses
- total_crop: total area with crops
- count_crop: count of different crops
- total_ncrop: total area with LULC classes that are not crop areas
- count_ncrop: count of areas that are not crops
- total_wood: total woodland area
- count_wood: count of woodland areas
- total_nwood: total area of no woodland area (the complementar set to woodland areas)
- count_nwood: count of no woodland areas (the complementar set to woodland areas)
- total_feed: total areas used by birds as feeding areas. includes, crops and woodland areas
- count_feed: count of total feeding areas
- total_tcrop: total area of temporary crops
- count_tcrop: count of total temporary crops
- total_pcrop: total area of permanent crops
- count_pcrop: count of total permanent crops


In [7]:
codes = '../external_data/CroplandCROS/CDL_code_names_colors.csv'
codes = pd.read_csv(codes)

In [8]:
codes.head(255)

,Codes,Class_Names,Scientific_name,EPPO_code,EPPO_code_period,Comments,IsCrop,IsCropOther,IsWoodland,IsFeedingArea,IsPerennial,IsTempCrop,IsPermCrop,Erdas_Red,Erdas_Green
0,0,Background,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0.0000,0.0000
1,1,Corn,Zea mays,ZEAMX,ZEAMX,NaN,1,1,0,1,0,1,0,1.0000,0.8275
2,2,Cotton,Gossypium spp.,1GOSG,1GOSG,NaN,1,1,0,1,0,1,0,1.0000,0.1490
3,3,Rice,Oryza sativa,ORYSA,ORYSA,NaN,1,1,0,1,0,1,0,0.0000,0.6588
4,4,Sorghum,Sorghum bicolor,SORVU,SORVU,NaN,1,1,0,1,0,1,0,1.0000,0.6196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,Cranberries,Vaccinium macrocarpon,VACMA,VACMA,NaN,1,1,0,1,1,0,1,1.0000,0.4000
251,251,,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1.0000,0.8275
252,252,,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0.1490,0.4431
253,253,,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0.6471,0.4392


In [9]:
# get codes values for crops
# codes = '/home/rfigueira/Documents/projectos/TainanMessina/article2/external_data/CroplandCROS/CDL_code_names_colors.csv'

# codes = pd.read_csv(codes)

# Filter cover codes that are crops or woodland
crop = codes[codes['IsCrop'] == 1]
ncrop = codes[codes['IsCrop'] == 0]
wood = codes[codes['IsWoodland'] == 1]
nwood = codes[codes['IsWoodland'] == 0]
feed = codes[codes['IsFeedingArea'] == 1]
perennial = codes[codes['IsPerennial'] == 1]
nperennial = codes[codes['IsPerennial'] == 0]
perm_crop = codes[codes['IsPermCrop'] == 1]
temp_crop = codes[codes['IsTempCrop'] == 1]

#  Convert to list of crop codes
check_crop = crop['Codes'].tolist()
check_ncrop = ncrop['Codes'].tolist()
check_wood = wood['Codes'].tolist()
check_nwood = nwood['Codes'].tolist()
check_feed = feed['Codes'].tolist()
check_perennial = perennial['Codes'].tolist()
check_nper = nperennial['Codes'].tolist()
check_perm_crop = perm_crop['Codes'].tolist()
check_temp_crop = temp_crop['Codes'].tolist()

In [10]:
# function to calculate Shannon entropy for each point. Adpated from original code created by Chat GPT 

import math

def calculate_shannon_entropy(dictionary):
    total_count = sum(dictionary.values())
    entropy = 0.0
    
    for count in dictionary.values():
        probability = count / total_count
        entropy -= probability * math.log2(probability)
    
    return entropy

In [11]:
# create a function that returns percentages and areas for each crop, in a dictionary

def calculate_percentage(data, keys_c, keys_nc, keys_w, keys_nw, keys_f, keys_pn, keys_npn, keys_p, keys_t):
    # convert string to dictionary
    d = eval(data)
    
    # Create a new dictionary with only the keys that are present in the list
    filtered_dict_c = {k: v for k, v in d.items() if k in keys_c}   # crops
    filtered_dict_nc = {k: v for k, v in d.items() if k in keys_nc} # no crops
    filtered_dict_w = {k: v for k, v in d.items() if k in keys_w}   # woodland
    filtered_dict_nw = {k: v for k, v in d.items() if k in keys_nw} # no woodland
    filtered_dict_f = {k: v for k, v in d.items() if k in keys_f} # feeding area
    filtered_dict_pn = {k: v for k, v in d.items() if k in keys_pn} # perennial
    filtered_dict_npn = {k: v for k, v in d.items() if k in keys_npn} # no perennial
    filtered_dict_p = {k: v for k, v in d.items() if k in keys_p}   # permanent crop
    filtered_dict_t = {k: v for k, v in d.items() if k in keys_t}   # temporary crop
    
    
    # Calculate the total number of pixels in the whole buffer
    total = sum(d.values()) # total area
    count_area = len(d.values()) # count of all land uses
    total_crop = sum(filtered_dict_c.values()) # total crop area
    count_crop = len(filtered_dict_c.values()) # count of crop
    total_ncrop = sum(filtered_dict_nc.values()) # total no crop area
    count_ncrop = len(filtered_dict_nc.values()) # count no crop areas
    total_wood = sum(filtered_dict_w.values()) # total woodland area
    count_wood = len(filtered_dict_w.values()) # count woodland areas
    total_nwood = sum(filtered_dict_nw.values()) # total no woodland area
    count_nwood = len(filtered_dict_nw.values()) # count no woodland areas
    total_feed = sum(filtered_dict_f.values()) # total feeding area
    count_feed = len(filtered_dict_f.values()) # count total feeding areas
    total_tcrop = sum(filtered_dict_t.values()) # total temporary crop area
    count_tcrop = len(filtered_dict_t.values()) # count temporary crops
    total_pcrop = sum(filtered_dict_p.values()) # total no woodland area
    count_pcrop = len(filtered_dict_p.values()) # count permanent crops
    
    # Calculate the fraction of the area for each value item that remained in the dictionary
    fract_c = {k: v/total for k, v in filtered_dict_c.items()}  # crop
    fract_nc = {k: v/total for k, v in filtered_dict_nc.items()} # no crop
    fract_w = {k: v/total for k, v in filtered_dict_w.items()}  # woodland 
    fract_nw = {k: v/total for k, v in filtered_dict_nw.items()} # no woodland 
    fract_f = {k: v/total for k, v in filtered_dict_f.items()} # feeding area 
    fract_pn = {k: v/total for k, v in filtered_dict_pn.items()} # perennial
    fract_npn = {k: v/total for k, v in filtered_dict_npn.items()} # no perennial
    if total_pcrop != 0:
        fract_p = {k: v/total_pcrop for k, v in filtered_dict_p.items()} # permanent crop
    else:
        fract_p = {k: 0 for k, v in filtered_dict_p.items()} # permanent crop
    if total_tcrop != 0:
        fract_t = {k: v/total_tcrop for k, v in filtered_dict_t.items()} # temporary crop
    else:
        fract_t = {k: 0 for k, v in filtered_dict_t.items()} # temporary crop
    area = {k: v*900/10**4 for k, v in d.items()} 
    area_crop = {k: v*900/10**4 for k, v in filtered_dict_c.items()}
    area_ncrop = {k: v*900/10**4 for k, v in filtered_dict_nc.items()}
    area_wood = {k: v*900/10**4 for k, v in filtered_dict_w.items()}
    area_nwood = {k: v*900/10**4 for k, v in filtered_dict_nw.items()}
    area_feed = {k: v*900/10**4 for k, v in filtered_dict_f.items()}
    area_tcrop = {k: v*900/10**4 for k, v in filtered_dict_t.items()}
    area_pcrop = {k: v*900/10**4 for k, v in filtered_dict_p.items()}
    
    #calculate buffer entropy for feeding pixels
    entropy = calculate_shannon_entropy(filtered_dict_f)
    
    return fract_c, fract_nc, fract_w, fract_nw, fract_f, fract_pn, fract_npn, fract_p, fract_t, entropy, \
        area, count_area, area_crop, count_crop, area_ncrop, count_ncrop, area_wood, count_wood, \
        area_nwood, count_nwood, area_feed, count_feed, area_tcrop, count_tcrop, area_pcrop, count_pcrop, \
        total*900/10**4, total_crop*900/10**4, total_ncrop*900/10**4, total_wood*900/10**4, \
        total_nwood*900/10**4, total_feed*900/10**4, total_tcrop*900/10**4, total_pcrop*900/10**4

The following executes the step of adding to the occurrence table several columns with the fraction (in relation to the unit) that each LULC represents, the area in hectars and the number of areas.

In [12]:
# add columns with percentage and area dictionaries to the dataframe

bird_data_name['fract_c'], bird_data_name['fract_nc'], bird_data_name['fract_w'], bird_data_name['fract_nw'], \
bird_data_name['fract_f'], bird_data_name['fract_pn'], bird_data_name['fract_npn'], bird_data_name['fract_p'], \
bird_data_name['fract_t'], bird_data_name['entr_f_area'], \
bird_data_name['area'], bird_data_name['count_area'], \
bird_data_name['area_crop'], bird_data_name['count_crop'], \
bird_data_name['area_ncrop'], bird_data_name['count_ncrop'], \
bird_data_name['area_wood'], bird_data_name['count_wood'], \
bird_data_name['area_nwood'], bird_data_name['count_nwood'], \
bird_data_name['area_feed'], bird_data_name['count_feed'], \
bird_data_name['area_tcrop'], bird_data_name['count_tcrop'], \
bird_data_name['area_pcrop'], bird_data_name['count_pcrop'], \
bird_data_name['total_area'], bird_data_name['total_crop_area'], \
bird_data_name['total_ncrop_area'], bird_data_name['total_wood_area'], bird_data_name['total_nwood_area'], \
bird_data_name['total_feed_area'], bird_data_name['total_temp_area'], bird_data_name['total_perm_area'] = \
zip(*bird_data_name.apply(lambda x: calculate_percentage(x['stats_cover'], \
                                                         check_crop, check_ncrop, check_wood, check_nwood, 
                                                         check_feed, \
                                                         check_perennial, check_nper, check_perm_crop, \
                                                         check_temp_crop), axis=1))


In [13]:
# export for control
bird_data_name.to_csv('../process_data/eBird_area_data_name.csv')
bird_data_name.head(40).to_csv('../process_data/eBird_area_data_name_head.csv')

In [14]:
# create a function to replace pixel codes by eppo codes 
def add_eppo_codes(a, b):
    new_dict = {}
    for key in a:
        new_dict[b[key]] = a[key]
    return new_dict

In [15]:
#  create dictionary for eppo codes
eppo = zip(crop['Codes'], crop['EPPO_code_period'])
eppo = dict(eppo)

In [16]:
%%time
# add columns with eppo:percentage and area to the dataframe
bird_data_name['fract_c_e'] = bird_data_name.apply(lambda x: add_eppo_codes(x['fract_c'], eppo), axis=1)
bird_data_name['fract_t_e'] = bird_data_name.apply(lambda x: add_eppo_codes(x['fract_t'], eppo), axis=1)
bird_data_name['fract_p_e'] = bird_data_name.apply(lambda x: add_eppo_codes(x['fract_p'], eppo), axis=1)

CPU times: user 2.13 s, sys: 281 ms, total: 2.41 s
Wall time: 2.42 s


In [17]:
# save the file
out_file = '../process_data/eBird_area_data_percent.csv'
bird_data_name.to_csv(out_file)

## 3.  Determine if there is consumption of pests by birds

In order to happen consumption of pests by bird, it is necessary to verify:
- if the crop to which a pest is related occurs in the buffer are of the occurrence point
- if the crop is temporary or permanent, and if the foraging niche trait of the bird species includes that type of habitat
- if the bird eats that pest

The next code does the following:

- get list of crops for the species occurrence, at buffer level
- check if bird predates a pest of that crop
    - first, get pests for the crop
    - second, check if bird preys the pest
    
It is added to the occurrence table a dictionary the combination of crop and pests (species and order) that the bird species of each occurrence predates. 

In [18]:
# load table with information about bird eats pests of a crop
bird_prey_crop = pd.read_csv('../process_data/bird_prey_crop.csv', low_memory=False)

In [19]:
bird_prey_crop.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'eppocode kingdom', 'eppocode phylum',
       'eppocode subphylum', 'eppocode class', 'eppocode order',
       'eppocode suborder', 'eppocode family', 'eppocode genus',
       'eppocode species', 'prefname kingdom', 'prefname phylum',
       'prefname subphylum', 'prefname class', 'prefname order',
       'prefname suborder', 'prefname family', 'prefname genus',
       'prefname species', 'eppocode', 'crop_eppocode', 'SCIENTIFIC NAME_x',
       'eats'],
      dtype='object')

In [20]:
# function to determine if bird preys a pest of a crop in the buffer

def bird_consum(bird, crops):
    df = pd.DataFrame()
    # set the filter conditions and copy to a new df
    df = bird_prey_crop.loc[ \
       (bird_prey_crop['SCIENTIFIC NAME_x'] == bird) & \
       (bird_prey_crop['crop_eppocode'].isin(crops)) & \
       (bird_prey_crop['eats'] == 1)]
    # get columns to return
    result = df[['crop_eppocode', 'eppocode']].values.tolist()
    result_order = df[['crop_eppocode', 'eppocode order']].values.tolist()
    return result, result_order

The next code adds to the table the columns `consum_t` and `consum_t_ord`. These columns have information about which temporary crops and pests are utilised by birds, for each point buffer. This is added at species and order level, for pests.

Afterwards, the same is done for permanent crops.

In [21]:
%%time
# Add columns to the occurrence table, for temp crops, adding a column with pairs (crop, pest), 10 min run
bird_data_name['consum_t'], bird_data_name['consum_t_ord'] = zip(*bird_data_name.apply( \
                                                  lambda x: bird_consum(x['SCIENTIFIC NAME_x'], x['fract_t_e']), \
                                                  axis=1))

CPU times: user 14min 45s, sys: 1.2 s, total: 14min 46s
Wall time: 14min 48s


In [22]:
%%time
# Add columns to the occurrence table, for perm crops, adding a column with pairs (crop, pest), 10 min run
bird_data_name['consum_p'], bird_data_name['consum_p_ord'] = zip(*bird_data_name.apply( \
                                                  lambda x: bird_consum(x['SCIENTIFIC NAME_x'], x['fract_p_e']), \
                                                  axis=1))

CPU times: user 14min 5s, sys: 800 ms, total: 14min 6s
Wall time: 14min 7s


In [23]:
bird_data_name

,Unnamed: 0.1,Unnamed: 0,GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER_x,CATEGORY,TAXON CONCEPT ID_x,COMMON NAME,SCIENTIFIC NAME_x,SUBSPECIES COMMON NAME,...,total_feed_area,total_temp_area,total_perm_area,fract_c_e,fract_t_e,fract_p_e,consum_t,consum_t_ord,consum_p,consum_p_ord
0,0,0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,2022-06-02 18:22:45.475308,26451,species,avibase-603194D3,Bewick's Wren,Thryomanes bewickii,NaN,...,189.09,0.63,0.72,"{'AVESA': 0.0032258064516129032, 'VITSS': 0.00...",{'AVESA': 1.0},{'VITSS': 1.0},"[[AVESA, DIABUH], [AVESA, HELIZE], [AVESA, MEL...","[[AVESA, 1COLEO], [AVESA, 1LEPIO], [AVESA, 1CO...","[[VITSS, DIABVZ], [VITSS, PLAAST], [VITSS, TOR...","[[VITSS, 1COLEO], [VITSS, 1LEPIO], [VITSS, 1LE..."
1,1,1926,URN:CornellLabOfOrnithology:EBIRD:OBS1446126014,2022-06-02 18:22:45.475308,11494,species,avibase-20C2214E,American Kestrel,Falco sparverius,NaN,...,189.09,0.63,0.72,"{'AVESA': 0.0032258064516129032, 'VITSS': 0.00...",{'AVESA': 1.0},{'VITSS': 1.0},"[[AVESA, HETDZE], [AVESA, DITYDI], [AVESA, MEL...","[[AVESA, 1RHABO], [AVESA, 1RHABO], [AVESA, 1RH...","[[VITSS, MELGET], [VITSS, DIABVZ], [VITSS, PLA...","[[VITSS, 1RHABO], [VITSS, 1COLEO], [VITSS, 1LE..."
2,2,2608,URN:CornellLabOfOrnithology:EBIRD:OBS1446126023,2022-06-02 18:22:45.475308,20505,species,avibase-151C2B3F,California Scrub-Jay,Aphelocoma californica,NaN,...,189.09,0.63,0.72,"{'AVESA': 0.0032258064516129032, 'VITSS': 0.00...",{'AVESA': 1.0},{'VITSS': 1.0},"[[AVESA, DIABUH], [AVESA, HELIZE], [AVESA, MEL...","[[AVESA, 1COLEO], [AVESA, 1LEPIO], [AVESA, 1CO...","[[VITSS, DIABVZ], [VITSS, PLAAST], [VITSS, TOR...","[[VITSS, 1COLEO], [VITSS, 1LEPIO], [VITSS, 1LE..."
3,3,9726,URN:CornellLabOfOrnithology:EBIRD:OBS1446126018,2022-06-02 18:22:45.475308,11371,species,avibase-75600969,Northern Flicker,Colaptes auratus,NaN,...,189.09,0.63,0.72,"{'AVESA': 0.0032258064516129032, 'VITSS': 0.00...",{'AVESA': 1.0},{'VITSS': 1.0},"[[AVESA, DIABUH], [AVESA, HELIZE], [AVESA, MEL...","[[AVESA, 1COLEO], [AVESA, 1LEPIO], [AVESA, 1CO...","[[VITSS, DIABVZ], [VITSS, PLAAST], [VITSS, TOR...","[[VITSS, 1COLEO], [VITSS, 1LEPIO], [VITSS, 1LE..."
4,4,10252,URN:CornellLabOfOrnithology:EBIRD:OBS1446126013,2022-06-02 18:22:45.475308,20762,species,avibase-9E9F2476,American Crow,Corvus brachyrhynchos,NaN,...,189.09,0.63,0.72,"{'AVESA': 0.0032258064516129032, 'VITSS': 0.00...",{'AVESA': 1.0},{'VITSS': 1.0},"[[AVESA, DIABUH], [AVESA, HELIZE], [AVESA, MEL...","[[AVESA, 1COLEO], [AVESA, 1LEPIO], [AVESA, 1CO...","[[VITSS, DIABVZ], [VITSS, PLAAST], [VITSS, TOR...","[[VITSS, 1COLEO], [VITSS, 1LEPIO], [VITSS, 1LE..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78937,78937,78917,URN:CornellLabOfOrnithology:EBIRD:OBS394851198,2021-03-31 02:38:15.410139,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,NaN,...,297.27,13.68,89.01,"{'ZEAMX': 0.0002870264064293915, 'ORYSA': 0.02...","{'ZEAMX': 0.006578947368421052, 'ORYSA': 0.631...","{'PRNPS': 0.005055611729019211, 'VITSS': 0.002...",[],[],[],[]
78938,78938,78918,URN:CornellLabOfOrnithology:EBIRD:OBS394868806,2021-03-31 02:38:15.410139,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,NaN,...,297.27,13.68,89.01,"{'ZEAMX': 0.0002870264064293915, 'ORYSA': 0.02...","{'ZEAMX': 0.006578947368421052, 'ORYSA': 0.631...","{'PRNPS': 0.005055611729019211, 'VITSS': 0.002...",[],[],[],[]
78939,78939,78919,URN:CornellLabOfOrnithology:EBIRD:OBS395381323,2021-03-31 02:38:15.410139,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,NaN,...,297.27,13.68,89.01,"{'ZEAMX': 0.0002870264064293915, 'ORYSA': 0.02...","{'ZEAMX': 0.006578947368421052, 'ORYSA': 0.631...","{'PRNPS': 0.005055611729019211, 'VITSS': 0.002...",[],[],[],[]
78940,78940,78922,URN:CornellLabOfOrnithology:EBIRD:OBS320781051,2015-05-17 23:58:28,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,NaN,...,305.01,0.09,2.07,"{'MEDSA': 0.00028694404591104734, 'MAB

In [24]:
# save table eBird_area_data_percent_step2.csv
bird_data_name.to_csv('../process_data/eBird_area_data_percent_step2.csv')

In [25]:
# function to retain area of crops for which birds provide service, and the cover percentage 

def keep_crop(consum, percent):
    cpr = [pair[0] for pair in consum]
    cpr = list(set(cpr))
    new_dict = {key: percent[key] for key in cpr if key in percent}
    return new_dict

The following add to the table a column that keeps only crops fior which birds provide service.

In [26]:
# apply to the table
# do for temporary crops
bird_data_name['keep_t'] = bird_data_name.apply( \
                                                  lambda x: keep_crop(x['consum_t'], x['fract_t_e']), \
                                                  axis=1)
# do for permanent crops
bird_data_name['keep_p'] = bird_data_name.apply( \
                                                  lambda x: keep_crop(x['consum_p'], x['fract_p_e']), \
                                                  axis=1)

In [27]:
# save table, eBird_area_data_percent-step3.csv
bird_data_name.to_csv('../process_data/eBird_area_data_percent-step3.csv')

In [28]:
bird_data_name

,Unnamed: 0.1,Unnamed: 0,GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER_x,CATEGORY,TAXON CONCEPT ID_x,COMMON NAME,SCIENTIFIC NAME_x,SUBSPECIES COMMON NAME,...,total_perm_area,fract_c_e,fract_t_e,fract_p_e,consum_t,consum_t_ord,consum_p,consum_p_ord,keep_t,keep_p
0,0,0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,2022-06-02 18:22:45.475308,26451,species,avibase-603194D3,Bewick's Wren,Thryomanes bewickii,NaN,...,0.72,"{'AVESA': 0.0032258064516129032, 'VITSS': 0.00...",{'AVESA': 1.0},{'VITSS': 1.0},"[[AVESA, DIABUH], [AVESA, HELIZE], [AVESA, MEL...","[[AVESA, 1COLEO], [AVESA, 1LEPIO], [AVESA, 1CO...","[[VITSS, DIABVZ], [VITSS, PLAAST], [VITSS, TOR...","[[VITSS, 1COLEO], [VITSS, 1LEPIO], [VITSS, 1LE...",{'AVESA': 1.0},{'VITSS': 1.0}
1,1,1926,URN:CornellLabOfOrnithology:EBIRD:OBS1446126014,2022-06-02 18:22:45.475308,11494,species,avibase-20C2214E,American Kestrel,Falco sparverius,NaN,...,0.72,"{'AVESA': 0.0032258064516129032, 'VITSS': 0.00...",{'AVESA': 1.0},{'VITSS': 1.0},"[[AVESA, HETDZE], [AVESA, DITYDI], [AVESA, MEL...","[[AVESA, 1RHABO], [AVESA, 1RHABO], [AVESA, 1RH...","[[VITSS, MELGET], [VITSS, DIABVZ], [VITSS, PLA...","[[VITSS, 1RHABO], [VITSS, 1COLEO], [VITSS, 1LE...",{'AVESA': 1.0},{'VITSS': 1.0}
2,2,2608,URN:CornellLabOfOrnithology:EBIRD:OBS1446126023,2022-06-02 18:22:45.475308,20505,species,avibase-151C2B3F,California Scrub-Jay,Aphelocoma californica,NaN,...,0.72,"{'AVESA': 0.0032258064516129032, 'VITSS': 0.00...",{'AVESA': 1.0},{'VITSS': 1.0},"[[AVESA, DIABUH], [AVESA, HELIZE], [AVESA, MEL...","[[AVESA, 1COLEO], [AVESA, 1LEPIO], [AVESA, 1CO...","[[VITSS, DIABVZ], [VITSS, PLAAST], [VITSS, TOR...","[[VITSS, 1COLEO], [VITSS, 1LEPIO], [VITSS, 1LE...",{'AVESA': 1.0},{'VITSS': 1.0}
3,3,9726,URN:CornellLabOfOrnithology:EBIRD:OBS1446126018,2022-06-02 18:22:45.475308,11371,species,avibase-75600969,Northern Flicker,Colaptes auratus,NaN,...,0.72,"{'AVESA': 0.0032258064516129032, 'VITSS': 0.00...",{'AVESA': 1.0},{'VITSS': 1.0},"[[AVESA, DIABUH], [AVESA, HELIZE], [AVESA, MEL...","[[AVESA, 1COLEO], [AVESA, 1LEPIO], [AVESA, 1CO...","[[VITSS, DIABVZ], [VITSS, PLAAST], [VITSS, TOR...","[[VITSS, 1COLEO], [VITSS, 1LEPIO], [VITSS, 1LE...",{'AVESA': 1.0},{'VITSS': 1.0}
4,4,10252,URN:CornellLabOfOrnithology:EBIRD:OBS1446126013,2022-06-02 18:22:45.475308,20762,species,avibase-9E9F2476,American Crow,Corvus brachyrhynchos,NaN,...,0.72,"{'AVESA': 0.0032258064516129032, 'VITSS': 0.00...",{'AVESA': 1.0},{'VITSS': 1.0},"[[AVESA, DIABUH], [AVESA, HELIZE], [AVESA, MEL...","[[AVESA, 1COLEO], [AVESA, 1LEPIO], [AVESA, 1CO...","[[VITSS, DIABVZ], [VITSS, PLAAST], [VITSS, TOR...","[[VITSS, 1COLEO], [VITSS, 1LEPIO], [VITSS, 1LE...",{'AVESA': 1.0},{'VITSS': 1.0}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78937,78937,78917,URN:CornellLabOfOrnithology:EBIRD:OBS394851198,2021-03-31 02:38:15.410139,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,NaN,...,89.01,"{'ZEAMX': 0.0002870264064293915, 'ORYSA': 0.02...","{'ZEAMX': 0.006578947368421052, 'ORYSA': 0.631...","{'PRNPS': 0.005055611729019211, 'VITSS': 0.002...",[],[],[],[],{},{}
78938,78938,78918,URN:CornellLabOfOrnithology:EBIRD:OBS394868806,2021-03-31 02:38:15.410139,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,NaN,...,89.01,"{'ZEAMX': 0.0002870264064293915, 'ORYSA': 0.02...","{'ZEAMX': 0.006578947368421052, 'ORYSA': 0.631...","{'PRNPS': 0.005055611729019211, 'VITSS': 0.002...",[],[],[],[],{},{}
78939,78939,78919,URN:CornellLabOfOrnithology:EBIRD:OBS395381323,2021-03-31 02:38:15.410139,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,NaN,...,89.01,"{'ZEAMX': 0.0002870264064293915, 'ORYSA': 0.02...","{'ZEAMX': 0.006578947368421052, 'ORYSA': 0.631...","{'PRNPS': 0.005055611729019211, 'VITSS': 0.002...",[],[],[],[],{},{}
78940,78940,78922,URN:CornellLabOfOrnithology:EBIRD:OBS320781051,2015-05-17 23:58:28,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,NaN,...,2.07,